### Imports e definições

In [43]:
import numpy as np
import pandas as pd

import warnings 
warnings.simplefilter(action="ignore")

ar_file = "qualidade_do_ar.csv"
bairros_file = "limite_de_bairros.csv"

# bairros monitorados
# já está ordenado para o merge entre as tabelas
bairros_names = (
    "Copacabana", 
    "Bangu", 
    "Centro", 
    "Campo Grande",
    "Irajá", 
    "Pedra de Guaratiba"
    "São Cristóvão", 
    "Tijuca", 
)

medidas_names = (
    'chuva',
    'pres',
    'rs',
    'temp',
    'ur',
    'dir_vento',
    'vel_vento',
    'so2',
    'no2',
    'hcnm',
    'hct',
    'ch4',
    'co',
    'no',
    'nox',
    'o3',
    'pm10',
    'pm2_5'
)

### Abrindo os arquivos

In [44]:
ar = pd.read_csv(ar_file)
bairros = pd.read_csv(bairros_file)

### Função para gerar a dimensão local

In [45]:
def create_dlocal(bairros_df: pd.DataFrame = bairros, ar_df: pd. DataFrame = ar) -> pd.DataFrame:
    """Cria a dimensão local"""
    
    # filtra o df pelos bairros que foram monitorados
    bairros_df = bairros_df[bairros_df["nome"].isin(bairros_names)]
    
    # pega somente as colunas de interesse
    columns = ["área", "nome", "regiao_adm", "codbairro", "codra", "codbnum", "rp", "cod_rp", "st_areashape", "st_perimetershape"]
    bairros_df = bairros_df[columns]
    
    # adiciona os nomes dos bairros ao df do monitoramento do ar
    data = dict(
    nome = ["Copacabana", "Bangu", "Centro", "Campo Grande","Irajá", "Pedra de Guaratiba", "São Cristóvão", "Tijuca"],
    codnum = [1, 2, 3, 4, 5, 6, 7, 8]
    )
    df = pd.DataFrame(data)
    ar_df = ar_df.merge(df, on="codnum", how="inner")
    
    # faz o merge do df de monitoramento do ar e dos bairros
    dlocal = ar_df.merge(bairros_df, on="nome", how="inner")
    
    # adiciona mais colunas de interesse
    columns.extend(["codnum", "lat", "lon", "x_utm_sirgas2000", "y_utm_sirgas2000"])
    # pega somente as colunas de interesse
    dlocal = dlocal[columns]
    
    return dlocal.drop_duplicates()

### Função para criar a tabela fato, dimensão medição

In [46]:
def create_dmedicao(ar_df: pd.DataFrame = ar, dlocal: pd.DataFrame = None) -> pd.DataFrame:
    """Cria a dimensão medição"""
    # colunas de interesse
    columns = [
        'data',
        'objectid',
        'codnum',
        'estação',
        'chuva',
        'pres',
        'rs',
        'temp',
        'ur',
        'dir_vento',
        'vel_vento',
        'so2',
        'no2',
        'hcnm',
        'hct',
        'ch4',
        'co',
        'no',
        'nox',
        'o3',
        'pm10',
        'pm2_5'
    ]
    
    # filtra pelas colunas de interesse
    dmedicao = ar_df[columns]
    # adiciona a coluna NOME no df
    dmedicao = dmedicao.merge(dlocal[["codnum", "nome"]].drop_duplicates(), on="codnum", how="inner")
    dmedicao["data"] = pd.to_datetime(dmedicao["data"])
    dmedicao["dia"] = dmedicao["data"].dt.date
    
    return dmedicao

### Função para criar a dimensão tempo

In [47]:
def create_dtempo(start = "2011-01-01", end = "2021-12-31") -> pd.DataFrame:
    """Cria a dimensão tempo"""
    
    date_range = pd.date_range(start=start, end=end)
    
    tempo_df = pd.DataFrame(dict(
        data = date_range,
        ano = date_range.year,
        mes = date_range.month, 
        dia = date_range.day, 
        dia_util = date_range.dayofweek
    ))
    
    tempo_df["dia_util"] = np.where(tempo_df["dia_util"] < 5, False, True)
    return tempo_df

In [48]:
dtempo = create_dtempo()
dlocal = create_dlocal()
dmedicao = create_dmedicao(dlocal=dlocal)

In [49]:
def create_mean_df(dmedicao: pd.DataFrame = dmedicao):
    groups = dmedicao.groupby([dmedicao["data"].dt.date, "codnum"])
    means_df = groups[['pm10', 'pm2_5', 'so2']].mean().reset_index()
    max_df = groups[["no2"]].max().reset_index()
    
    return groups, means_df, max_df

In [50]:
# Função para calcular a maior média móvel de 8 horas
def maior_media_movel(grupo):
    grupo = grupo.set_index('data').resample('1H').mean()
    media_movel = grupo[['o3','co']].rolling(window=8).mean()
    return media_movel.max()

df_8h = dmedicao.groupby(['codnum', 'dia']).apply(maior_media_movel).reset_index()

In [51]:
groups, means_df, max_df = create_mean_df()

In [52]:
df = iter(groups)

In [53]:
_, df_ = next(df)
# df_

In [54]:
means_df[means_df["codnum"] == 1]

,data,codnum,pm10,pm2_5,so2
0,2011-01-01,1,36.500476,NaN,1.330952
3,2011-01-02,1,34.262083,NaN,1.725652
6,2011-01-03,1,43.686667,NaN,4.057826
9,2011-01-04,1,43.392917,NaN,2.953913
12,2011-01-05,1,46.662500,NaN,4.355652
...,...,...,...,...,...
20084,2020-10-25,1,53.529167,NaN,NaN
20090,2020-10-26,1,49.292727,NaN,NaN
20096,2020-10-27,1,59.924583,NaN,NaN
20102,2020-10-28,1,105.077368,NaN,NaN


In [55]:
max_df[max_df["no2"].notna()]

,data,codnum,no2
1094,2012-01-01,2,28.83
1096,2012-01-01,4,49.63
1097,2012-01-01,5,56.62
1100,2012-01-02,2,27.65
1102,2012-01-02,4,42.70
...,...,...,...
22121,2021-12-31,8,24.43
22122,2022-01-01,2,14.44
22124,2022-01-01,4,36.35
22125,2022-01-01,5,23.03


In [67]:
# Definir as faixas de concentração e índices correspondentes
faixas = {
    'pm10': [(0, 50, 0, 40), (51, 100, 41, 80), (101, 150, 81, 120), (151, 250, 121, 200), (251, 600, 201, 400)],
    'pm2_5': [(0, 25, 0, 40), (26, 50, 41, 80), (51, 75, 81, 120), (76, 125, 121, 200), (126, 300, 201, 400)],
    'o3': [(0, 100, 0, 40), (101, 130, 41, 80), (131, 160, 81, 120), (161, 200, 121, 200), (201, 800, 201, 400)],
    'co': [(0, 9, 0, 40), (10, 11, 41, 80), (12, 13, 81, 120), (14, 15, 121, 200), (16, 50, 201, 400)],
    'no2': [(0, 200, 0, 40), (201, 240, 41, 80), (241, 320, 81, 120), (321, 1130, 121, 200), (1131, 3750, 201, 400)],
    'so2': [(0, 20, 0, 40), (21, 40, 41, 80), (41, 365, 81, 120), (366, 800, 121, 200), (801, 2620, 201, 400)]
}

# Função para calcular o IQAr para uma medição
def calcular_iqar(concentracao, faixa):
    for (c_ini, c_fin, i_ini, i_fin) in faixas[faixa]:
        if c_ini <= concentracao <= c_fin:
            iqar = i_ini + (i_fin - i_ini) / (c_fin - c_ini) * (concentracao - c_ini)
            return iqar
    return None

In [57]:
calcular_iqar(135, faixas["o3"])

86.37931034482759

In [58]:
df_pm10_pm2_5_so2 = dmedicao.groupby(['data', 'codnum'])[['pm10', 'pm2_5', 'so2']].mean().reset_index()

In [59]:
df_final = means_df.merge(max_df, on=["codnum", "data"], how="inner").merge(df_8h, right_on=["codnum", "dia"], left_on=["codnum", "data"], how="inner")[["data", "codnum", "pm10", "pm2_5", "so2", "no2", "o3", "co"]]

In [71]:
for col in ["pm10", "pm2_5", "so2", "no2", "o3", "co"]:
    df_final[f"{col}_i"] = df_final[col].apply(calcular_iqar, args=(col,))

In [72]:
df_final

,data,codnum,pm10,pm2_5,so2,no2,o3,co,pm10_i,pm2_5_i,so2_i,no2_i,o3_i,co_i
0,2011-01-01,1,36.500476,NaN,1.330952,NaN,32.3100,0.14375,29.200381,NaN,2.661905,NaN,12.924,0.638889
1,2011-01-01,3,14.666667,NaN,NaN,NaN,43.0100,0.28375,11.733333,NaN,NaN,NaN,17.204,1.261111
2,2011-01-01,8,18.237143,NaN,1.296316,NaN,60.5200,0.10250,14.589714,NaN,2.592632,NaN,24.208,0.455556
3,2011-01-02,1,34.262083,NaN,1.725652,NaN,16.9975,0.46500,27.409667,NaN,3.451304,NaN,6.799,2.066667
4,2011-01-02,3,17.000000,NaN,NaN,NaN,33.8625,0.37250,13.600000,NaN,NaN,NaN,13.545,1.655556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22122,2022-01-01,2,20.333333,NaN,7.306667,14.44,NaN,NaN,16.266667,NaN,14.613333,2.888,NaN,NaN
22123,2022-01-01,3,30.333333,NaN,NaN,NaN,NaN,NaN,24.266667,NaN,NaN,NaN,NaN,NaN
22124,2022-01-01,4,29.333333,NaN,7.903333,36.35,NaN,NaN,23.466667,NaN,15.806667,7.270,NaN,NaN
22125,2022-01-01,5,38.666667,26.333333,6.410000,23.03,NaN,NaN,30.933333,41.541667,12.820000,4.606,NaN,NaN


In [73]:
# for col in ["pm10", "pm2_5", "so2", "no2", "o3", "co"]:
#     df_final.loc[:, f"{col}_i"] = calcular_iqar(df_final[col].values, col)

In [75]:
df_final["max_value"] = df_final[["pm10_i",	"pm2_5_i", "so2_i",	"no2_i","o3_i",	"co_i"]].apply(max, axis=1)

In [76]:
df_final

,data,codnum,pm10,pm2_5,so2,no2,o3,co,pm10_i,pm2_5_i,so2_i,no2_i,o3_i,co_i,max_value
0,2011-01-01,1,36.500476,NaN,1.330952,NaN,32.3100,0.14375,29.200381,NaN,2.661905,NaN,12.924,0.638889,29.200381
1,2011-01-01,3,14.666667,NaN,NaN,NaN,43.0100,0.28375,11.733333,NaN,NaN,NaN,17.204,1.261111,17.204000
2,2011-01-01,8,18.237143,NaN,1.296316,NaN,60.5200,0.10250,14.589714,NaN,2.592632,NaN,24.208,0.455556,24.208000
3,2011-01-02,1,34.262083,NaN,1.725652,NaN,16.9975,0.46500,27.409667,NaN,3.451304,NaN,6.799,2.066667,27.409667
4,2011-01-02,3,17.000000,NaN,NaN,NaN,33.8625,0.37250,13.600000,NaN,NaN,NaN,13.545,1.655556,13.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22122,2022-01-01,2,20.333333,NaN,7.306667,14.44,NaN,NaN,16.266667,NaN,14.613333,2.888,NaN,NaN,16.266667
22123,2022-01-01,3,30.333333,NaN,NaN,NaN,NaN,NaN,24.266667,NaN,NaN,NaN,NaN,NaN,24.266667
22124,2022-01-01,4,29.333333,NaN,7.903333,36.35,NaN,NaN,23.466667,NaN,15.806667,7.270,NaN,NaN,23.466667
22125,2022-01-01,5,38.666667,26.333333,6.410000,23.03,NaN,NaN,30.933333,41.541667,12.820000,4.606,NaN,NaN,41.541667


In [94]:
def categorize_pm(value):
    if pd.isnull(value):
        return "aaa"
    if value >= 0 and value <= 40:
        return 'Boa'
    elif value >= 41 and value <= 80:
        return 'Moderada'
    elif value >= 81 and value <= 120:
        return 'Ruim'
    elif value >= 121 and value <= 200:
        return 'Muito Ruim'
    elif value >= 201 and value <= 400:
        return 'Péssima'
    else:
        return 'Indisponível'

In [101]:
df_final["qualidade"] = df_final["max_value"].apply(categorize_pm)

In [103]:
df_final.to_csv("valores.csv")

In [104]:
df_final

,data,codnum,pm10,pm2_5,so2,no2,o3,co,pm10_i,pm2_5_i,so2_i,no2_i,o3_i,co_i,max_value,qualidade
0,2011-01-01,1,36.500476,NaN,1.330952,NaN,32.3100,0.14375,29.200381,NaN,2.661905,NaN,12.924,0.638889,29.200381,Boa
1,2011-01-01,3,14.666667,NaN,NaN,NaN,43.0100,0.28375,11.733333,NaN,NaN,NaN,17.204,1.261111,17.204000,Boa
2,2011-01-01,8,18.237143,NaN,1.296316,NaN,60.5200,0.10250,14.589714,NaN,2.592632,NaN,24.208,0.455556,24.208000,Boa
3,2011-01-02,1,34.262083,NaN,1.725652,NaN,16.9975,0.46500,27.409667,NaN,3.451304,NaN,6.799,2.066667,27.409667,Boa
4,2011-01-02,3,17.000000,NaN,NaN,NaN,33.8625,0.37250,13.600000,NaN,NaN,NaN,13.545,1.655556,13.600000,Boa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22122,2022-01-01,2,20.333333,NaN,7.306667,14.44,NaN,NaN,16.266667,NaN,14.613333,2.888,NaN,NaN,16.266667,Boa
22123,2022-01-01,3,30.333333,NaN,NaN,NaN,NaN,NaN,24.266667,NaN,NaN,NaN,NaN,NaN,24.266667,Boa
22124,2022-01-01,4,29.333333,NaN,7.903333,36.35,NaN,NaN,23.466667,NaN,15.806667,7.270,NaN,NaN,23.466667,Boa
22125,2022-01-01,5,38.666667,26.333333,6.410000,23.03,NaN,NaN,30.933333,41.541667,12.820000,4.606,NaN,NaN,41.541667,Moderada


In [108]:
df_final = df_final.merge(dmedicao[["codnum", "nome"]].drop_duplicates(), on="codnum", how="inner")

In [110]:
df_final.to_csv("valores.csv")